# **Тренировка модели для задачи получения предсказания**

##**1. Импортируем библиотеки**

In [2]:
!pip install --upgrade pip
!pip install torch==1.13.1 \
    torchdata==0.5.1 --quiet
!pip install \
    transformers==4.27.2 \
    datasets==2.11.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.16.0 requires torchdata==0.7.0, but you have torchdata 0.5.1 which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━

In [3]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, Trainer, TrainingArguments
import torch
import time
import evaluate
import pandas as pd
import numpy as np

##**2. Загружаем датасет для fine-tuning тренировки**

In [4]:
from datasets import load_dataset
huggingface_dataset_name = "dkagramanyan/horoscopes_ru" #TODO: выбрать подходящий датасет
dataset = load_dataset(huggingface_dataset_name)
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/6976 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/66501 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/dkagramanyan___parquet/dkagramanyan--horoscopes_ru-3b8c7396bfa23654/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['date', 'sign', 'text'],
        num_rows: 6976
    })
    train: Dataset({
        features: ['date', 'sign', 'text'],
        num_rows: 66501
    })
})

**Подготовка датасета для входа модели**

In [5]:
#map делает операции сразу и над train, и над test
#загружаем токенайзер
tokenizer = AutoTokenizer.from_pretrained('AlexWortega/wortegaLM',padding_side='left')

tokenizer_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

In [8]:
def preprocessing(examples): #examples - это словарь с date, sign, text
    examples['sign'] = examples['sign'].replace("scorpio", "скорпион")
    examples['Prediction'] = f'Предсказание судьбы для знака зодиака {examples["sign"]} {examples["text"]}' #prompt
    tokens = tokenizer(examples['Prediction'], truncation=False, return_tensors=None)
    tokens['labels'] = tokens['input_ids'].copy()

    return tokens


In [11]:
dataset= dataset.map(
       lambda examples: preprocessing(examples),
       batched=False,
       remove_columns = dataset['train'].column_names
)



Map:   0%|          | 0/6976 [00:00<?, ? examples/s]

Map:   0%|          | 0/66501 [00:00<?, ? examples/s]

In [12]:
dataset['train'][0]

{'Prediction': 'Предсказание судьбы для знака зодиака овен Любые разногласия во мнениях скоро улягутся, а вы продолжайте делать, как делали, но постарайтесь не наступать на ноги слишком многим, иначе ваши сегодняшние действия сыграют против вас в будущем.',
 'input_ids': [2,
  3974,
  14,
  2995,
  8,
  12,
  1487,
  33,
  80,
  2355,
  11,
  5,
  92,
  13,
  55,
  8,
  11,
  66,
  5,
  13,
  10,
  12,
  30,
  5,
  4395,
  33,
  12,
  13798,
  24,
  141,
  5,
  6618,
  40,
  1090,
  5,
  3145,
  81,
  3387,
  6,
  5,
  11,
  64,
  3050,
  441,
  424,
  6,
  53,
  5,
  4668,
  6,
  74,
  7428,
  1598,
  21,
  23,
  4460,
  23,
  1519,
  698,
  6085,
  16,
  6,
  1439,
  2284,
  5,
  7301,
  12,
  1431,
  6641,
  48,
  19,
  397,
  270,
  5,
  10,
  5,
  3778,
  7,
  3],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  

##Грустные пытки датасета

In [ ]:
#TODO:  чтобы почистить данные можно и с объектом datasets работать через метод map. Токенайзеру можно подавать 1 текст
"""
#аналогично с тестовым датасетом, если он нужен?
df_train = pd.DataFrame.from_dict(dataset['train'])
df_train.drop('date', axis=1, inplace=True)
df_train['sign'] = df_train['sign'].replace('scorpio', 'скорпион') #сделала данные однородными в колонке sign
#делаем из датафрейма один большой текст
df_train['prediction'] = 'Предсказание судьбы для знака зодиака ' + df_train['sign'] + ' ' + df_train['text']
df_train = df_train.drop(['sign', 'text'], axis = 1)
#df_train = ' '.join(df_train['prediction'].tolist())
"""

In [ ]:
"""
https://github.com/huggingface/peft/blob/main/examples/causal_language_modeling/peft_prompt_tuning_clm.ipynb

SEq2Seq в input_ids мы храним только промтп, тексты и аттеншн маску, а labels - это input_ids результата, который мы хотим получить.
CausalLLM - input_ids равны labels (два идентичных массива, иначе смэрть)

"""
#Промпт + знак зодиака + предсказание, верно?
#def tokenize_function(df_train):
  #start_prompt = 'Предсказание судьбы для знака зодиака '
  #prompt = [start_prompt + sign + dialogue  for dialogue in example['dialogue']] #TODO: меняем относительно датасета
  #df_train['input_ids'] = tokenizer(df_train, padding="max_length", truncation=True, return_tensor="pt").input_ids #почему padding такой? паддинг добиваем нулями чтобы вектора были одного размера
  #truncation  - обрезает лишний текст. padding = longest - до макс длины в батче добьет, padding="max_length" - лучше когда torch-compile
  #example['labels'] = tokenizer(example['summary'], padding="max_length", truncation=True, return_tensor="pt").input_ids

df_train = ' '.join(df_train['prediction'].tolist())
model_input = tokenizer(df_train) #, padding="max_length", truncation=True)
#df_train['input_ids'] = tokenizer(df_train, padding="max_length", truncation=True).input_ids
#input_ids = tokenizer(df_train, return_tensors="pt").input_ids
#tokenizer_datasets = dataset.map(df_train, batched=True)
#tokenizer_datasets
#tokenizer_datasets = tokenizer_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary'])

In [ ]:
#загружаем токенайзер
tokenizer = AutoTokenizer.from_pretrained('AlexWortega/wortegaLM',padding_side='left')

# encode input context
input_ids = tokenizer(input_context, return_tensors="pt").input_ids

In [ ]:
#input_ids: list of token ids to be fed to a model
#смотрим разбивку на трейн тест и валидационную выборку
print('Shapes of the datasets:')
print(f'Training: {tokenizer_datasets['train'].shape}')
print(f'Validation: {tokenizer_datasets['validation'].shape}')
print(f'Test: {tokenizer_datasets['test'].shape}')

##**3. Загружаем модель**

In [13]:
#загружаем модель
model_name = AutoModelForCausalLM.from_pretrained('AlexWortega/wortegaLM')

model_name.resize_token_embeddings(len(tokenizer))
#device = 'cuda' в коллабе тоже нужно?
#model_name.to(device)

config.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/489M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Embedding(30000, 768)

In [ ]:
def generate_seqs(q, model, k=2):
    gen_kwargs = {
        "min_length": 20,
        "max_new_tokens": 100,
        "top_k": 50,
        "top_p": 0.7,
        "do_sample": True,
        "early_stopping": True,
        "no_repeat_ngram_size": 2,
        "eos_token_id": tokenizer.eos_token_id,
        "pad_token_id": tokenizer.eos_token_id,
        "use_cache": True,
        "repetition_penalty": 1.5,
        "length_penalty": 1.2,
        "num_beams": 4,
        "num_return_sequences": k
    }

    t = tokenizer.encode(q, add_special_tokens=False, return_tensors='pt') #.to(device)
    g = model_name.generate(t, **gen_kwargs)
    generated_sequences = tokenizer.batch_decode(g, skip_special_tokens=False)

    return  generated_sequences


In [ ]:
generate_seqs('Что бы мне поделать', model_name, k=2) #проверили, что моделька работает

['Что бы мне поделать в комментариях.\n## 1\\. Создаем следующий код:\n[code]\n #!/usr/local/bin/env --prefix="http://www.wikipedia.org/wiki/2019/XMLS_PATH" > /etc/apt/library/packages/downloads.conf\n',
 'Что бы мне поделать в 2016 году.\n## 3\\. Создаем интерфейс для работы с Bluetooth-адресом:\n[code]\n <?xml version="http://www.wikipedia.org/wiki/Spotify">\n import org.apache.com/downloads/virtual-tools.html\n']

##**5. Fine-tuning модели при помощи PEFT/LoRA**

In [ ]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 8.0 MB/s eta 0:00:00


In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=8,  #rank
    lora_alpha=16, #количество матриц уточнить
    target_modules=['q', 'v'], #формула селф аттеншн, изучить  (еще есть k, o) можно напечатать модель и посмотреть как ее слои называются
    lora_dropout=0.05,
    bias='none',
    task_type = TaskType.CAUSAL_LM #FLAN-T5 TODO: заменить CasualLLM (в Peft repo)
)

In [ ]:
#добавляем параметры к оригинальной LLM
peft_model = get_peft_model(model_name,
                            lora_config) #заменяет self-attention слой на лора адаптер

NameError: name 'model_name' is not defined

In [ ]:
output_dir = f'./peft-model-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir = output_dir,
    auto_find_batch_size = True,
    learning_rate=1e-3,
    num_train_epochs=1, #поэспериментировать
    logging_steps=1,
    max_steps=1
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset = model_input
)

In [ ]:
#тренируем и сохраняем результат
peft_trainer.train()

peft_model_path = "./peft-model-checkpoint-local"
peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

In [ ]:
#Генерируем предсказание с нужным текстом

generate_seqs('🔮 Гадалка Агата говорит, что', peft_model, k=1)

##**6. Готовим модель для инференса**

In [ ]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForCausalLM.from_pretrain('AlexWortega/wortegaLM', torch_dtype=torch.bfloat16) #torch_dtype убрать?
tokenizer = AutoTokenizer.from_pretrained('AlexWortega/wortegaLM')

peft_model = PeftModel.from_pretrained(peft_model_base,
                                       "./peft-model-checkpoint-local",
                                       torch_dtype=torch.bfloat16,
                                       is_trainable = False)

**7. Проверяем качество **